In [1]:
import sys
import os
workPath = os.path.abspath('..')
if not workPath in sys.path:
    sys.path.append(workPath)

In [3]:
class Space:
    def __init__(self):
        pass

In [14]:
myset = set([1, 2])

In [15]:
myset.intersection(set([2, 3]))

{2}

In [ ]:
sorted(set([1, 2]))

In [ ]:
#         if any([x is None for x in [system, avec, bvec]]):
#             raise TypeError
#         if avec.inputs == bvec.inputs:
#             raise UnderdefinedError
#         self.commonkeys = sorted(set(avec.keys).intersect(set(bvec.keys)))
#         if len(self.commonkeys) < 1:
#             raise UnderdefinedError

#         uncommonkeys = []
#         uncommonkeys.extend([key for key in avec.keys if not key in bvec.keys])
#         uncommonkeys.extend([key for key in bvec.keys if not key in avec.keys])
#         self.uncommonkeys = sorted(uncommonkeys)

#         self.keys = sorted([*avec.keys, *bvec.keys])

In [23]:
aset = set([1, 2, 3])
aset.symmetric_difference

In [24]:
set.symmetric_difference(set([1, 2, 3]), set([3, 4, 5]))

{1, 2, 4, 5}

In [ ]:

#         uncommonkeys = sorted(
#             set.symmetric_difference(
#                 set(avec.keys),
#                 set(bvec.keys)
#                 )
#             )

        
        
#         allkeys = set([*avec.keys, *bvec.keys])
#         pins = [*system.defaultInps.keys, *[key for key in ]


#         self.mins = {key: min(avec[key], bvec[key]) for key in self.commonkeys}
#         self.mins = {
#             **system.defaultInps,
#             **{key: min(avec[key], bvec[key]) for key in self.commonkeys}
#             }
#         self.maxs = {
#             **system.defaultInps,
#             **{key: min(avec[key], bvec[key]) for key in self.commonkeys}
#             }
#         self.varying = []
#         for key in avec.keys:
#             if key in bvec.keys:
#                 if not avec[key] == bvec[key]:
#                     self.varying.append(key)
#         self.data = np.vstack([avec.data, bvec.data])
#         self.system, self.avec, self.bvec = system, avec, bvec
#         super().__init__(**kwargs)
#     def _slice_vector(self, inVec):
#         dict1 = {**self.avec.inputs, **inVec.inputs}
#         dict2 = {**self.bvec.inputs, **inVec.inputs}
#         if dict1 == dict2:
#             return _slice_full_vector(inVec)
#         else:
#             return _slice_partial_vector(dict1, dict2)
#     def _slice_full_vector(self, inVec):
#         if not all([
#                 self.mins[key] <= inVec[key] <= self.maxs[key] \
#                      for key in inVec.keys
#                 ]):
#             raise OutOfSpaceError
#         if self.avec <= inVec <= self.bvec:
#             return system[inVec]
            
#     def _slice_partial_vector(self, dict1, dict2):
#         return self.__class__(
#             system = self.system,
#             avec = vector.build(**dict1),
#             bvec = vector.build(**dict2)
#             ) # UPDATE THIS!!! BUILD
#     def __getitem__(self, arg):
#         if isinstance(arg, vector.CLASS):
#             if arg.keys == self.keys:
#                 return self._slice_vector(arg)

In [30]:
def all_dict(*inDicts):
    outDict = dict()
    for inDict in inDicts:
        for key, val in inDict.items():
            if not key in outDict:
                outDict[key] = []
            outDict[key].append(val)
    for key, val in outDict.items():
        if len(val) == 1:
            newVal = val[0]
        else:
            newVal = sorted(set(val))
        outDict[key] = newVal
    return outDict

In [31]:
all_dict({'a': 1, 'b': 2}, {'b': 2, 'c': 3}, {'b': 3, 'c': 4})

{'a': 1, 'b': [2, 3], 'c': [3, 4]}

In [32]:
list([1, 2])

[1, 2]

In [ ]:
mins, maxs = list(map(list, zip(*[
    min(list(vals)), max(list(vals)) \
        for key, vals in sorted(allDict.items())
    ])))

In [37]:
list(map(list, zip(*[[1, 2], [3, 4]])))

[[1, 3], [2, 4]]

In [168]:
class HyperCube(Space):

    def __init__(
            self,
            avec = None,
            bvec = None,
            **kwargs
            ):

        self.avec, self.bvec = avec, bvec
        self.keys = sorted([*avec.keys, *bvec.keys])

    def _slice_vector(self, inVec):
        allkeys = sorted(set([*inVec.keys, *self.keys]))
        a, b, c = [], [], []
        for key in allkeys:
            for sublist, obj in zip((a, b, c), (self.avec, self.bvec, inVec)):
                if key in obj:
                    sublist.append(obj[key])
                else:
                    sublist.append(None)
        out1 = [None for i in allkeys]
        out2 = [None for i in allkeys]
        for index in range(len(allkeys)):
            aval, bval, cval = a[index], b[index], c[index]
            if cval is None:
                if (not aval is None) and (not bval is None):
                    out1[index], out2[index] = aval, bval
                elif not aval is None:
                    out1[index] = out2[index] = aval
                elif not bval is None:
                    out1[index] = out2[index] = bval
            elif (not aval is None) and (not bval is None):
                if min(aval, bval) <= cval <= max(aval, bval):
                    out1[index] = out2[index] = cval
                else:
                    raise OutOfSpaceError
            elif not aval is None:
                if cval == aval:
                    out1[index] = out2[index] = aval
                else:
                    raise OutOfSpaceError
            elif not bval is None:
                if cval == bval:
                    out1[index] = out2[index] = bval
                else:
                    raise OutOfSpaceError
            else:
                out1[index] = out2[index] = cval
        buildVecFn = inVec.__class__.build
        if out1 == out2:
            return buildVecFn(**dict(zip(allkeys, out1)))
        else:
            return self.__class__(
                buildVecFn(**dict(zip(allkeys, out1))),
                buildVecFn(**dict(zip(allkeys, out2)))
                )

    def __getitem__(self, arg):
        return self._slice_vector(arg)
        
#         checks = []
#         for key, val in inVec.inputs.items():
#             if key in self.keys:
#                 minVal, maxVal = dict(zip(keys, (mins, maxs)))[key]
#                 if minVal <= val <= maxVal:
#                     checks.append(True)
            

In [169]:
from everest.builts import vector
a = vector.build(foo = 2., bar = 20., baz = 7.)
b = vector.build(foo = 3., bar = 30., qux = 8.)
cube = HyperCube(avec = a, bvec = b)
c = vector.build(foo = 2.5, bar = 25., quux = 9.)
out = cube[c]
out

In [171]:
a = vector.build(foo = 2., bar = 20., baz = 7.)
b = vector.build(foo = 3., bar = 30., qux = 8.)
cube = HyperCube(avec = a, bvec = b)
c = vector.build(quux = 9.)
out = cube[c]
out

In [9]:
# import random
# class MyIterable:
#     def __iter__(self):
#         return MyIterator()
# class MyIterator:
#     myiter = iter(range(100))
#     def __next__(self):
#         return next(self.myiter)